# SMS-classification

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [34]:
mails = pd.read_csv("./DataSets/SMS-Text-Classification/SMS_train.csv",encoding='ISO-8859-1')
mails.head()

,S. No.,Message_body,Label
0,1,Rofl. Its true to its name,Non-Spam
1,2,The guy did some bitching but I acted like i'd...,Non-Spam
2,3,"Pity, * was in mood for that. So...any other s...",Non-Spam
3,4,Will ü b going to esplanade fr home?,Non-Spam
4,5,This is the 2nd time we have tried 2 contact u...,Spam


In [3]:
#get rows and columns size
mails.shape

(957, 3)

In [4]:
# get null values
mails.isnull().sum()

S. No.          0
Message_body    0
Label           0
dtype: int64

In [5]:
#check if there are duplicates
mails.duplicated().sum()

0

In [6]:
#lets check the way the first 4 rows looks
for each in mails['Message_body'].head(4):
    print("\n\n")
    print(each)




Rofl. Its true to its name



The guy did some bitching but I acted like i'd be interested in buying something else next week and he gave it to us for free



Pity, * was in mood for that. So...any other suggestions?



Will ü b going to esplanade fr home?


In [7]:
#check if we have urls
pattern = r"(?P<protocol>https?)://(?P<domain>[\w\.\-]+)/?(?P<path>.*)"
urls = mails['Message_body'].str.extract(pattern)

In [8]:
urls.sum()
#thre are no urls

Series([], dtype: float64)

In [9]:
#removing emails keywords
#we will replace with ""
import re

pattern = r"e[-\s]?mail"
email_uniform = mails['Message_body'].str.replace(pattern , "email" , flags=re.I)

In [10]:
#working with numbers
pattern =r"([0-9]+)"
nums = mails["Message_body"].str.extractall(pattern)



In [11]:
import string
import re
x = string.punctuation
pattern =r"([0-9]+)"
def clearAll(Message_body):
    Message_body = Message_body.replace(string.punctuation , "")
    
    return Message_body

In [12]:
mails.head(4).apply(clearAll)

,S. No.,Message_body,Label
0,1,Rofl. Its true to its name,Non-Spam
1,2,The guy did some bitching but I acted like i'd...,Non-Spam
2,3,"Pity, * was in mood for that. So...any other s...",Non-Spam
3,4,Will ü b going to esplanade fr home?,Non-Spam


In [13]:
# remove any non chars
for c in range(mails.shape[0]):
    mails['Message_body'].iloc[c] = [x for x in mails['Message_body'].iloc[c] if x not in string.punctuation]
    mails['Message_body'].iloc[c] =''.join(mails['Message_body'].iloc[c])

In [14]:
# pattern = r"([0-9]+)"
#remove the digits
number = ["1" , "2","3" ,'4' ,'5','6','7','8','9' ,"0"]
for c in  range(mails.shape[0]):
    mails['Message_body'].iloc[c] = [x for x in mails['Message_body'].iloc[c] if x not in number]
    mails['Message_body'].iloc[c] =''.join(mails['Message_body'].iloc[c])

In [15]:
#do vectorization and covert it to matrix
from sklearn.feature_extraction.text import CountVectorizer

vec = CountVectorizer()

text = vec.fit_transform(mails['Message_body'])


In [16]:
#get the labels
y = mails['Label']
text.shape


(957, 3022)

In [17]:
#do splitliting the data 80 to 20
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(text , y , test_size = 0.2 , random_state = 0)

In [18]:
#training using naibayes classifier(multinomial)
from sklearn.naive_bayes import MultinomialNB
clf1 = MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [19]:
#cross validation for naive bayes algo
from sklearn.model_selection import cross_val_score
valid = cross_val_score(estimator = clf1 , X = text, y=y , cv=10)
print("Cross Validation results for 10 folds")
print(valid)
print(f"Mean for cross val is {valid.mean()}")
print(f"std for the cross val is {valid.std()}")

Cross Validation results for 10 folds
[1.         0.95833333 0.96875    1.         0.94791667 0.92708333
 0.94791667 0.98947368 0.96842105 0.93684211]
Mean for cross val is 0.9644736842105264
std for the cross val is 0.02435287022912144


In [20]:
#do training baesd on multinomialnb
clf1.fit(x_train, y_train)

MultinomialNB()

In [21]:
#checking score based on train set
from sklearn.metrics import classification_report , accuracy_score , confusion_matrix
train_pred = clf1.predict(x_train)

print(f"Classification Repport\n  {classification_report(y_train , train_pred)}")

print("\n\n")
print(f"Confudion Matrix  \n {confusion_matrix(y_train ,train_pred)}")

print("\n\n")
print(f"Accuracy score   {accuracy_score(y_train ,train_pred)}")

Classification Repport
                precision    recall  f1-score   support

    Non-Spam       0.99      1.00      1.00       673
        Spam       0.99      0.96      0.97        92

    accuracy                           0.99       765
   macro avg       0.99      0.98      0.98       765
weighted avg       0.99      0.99      0.99       765




Confudion Matrix  
 [[672   1]
 [  4  88]]



Accuracy score   0.9934640522875817


In [22]:
#checking score based on set test
pred = clf1.predict(x_test)

print(f"Classification Repport\n  {classification_report(y_test , pred)}")

print("\n\n")
print(f"Confudion Matrix  \n {confusion_matrix(y_test , pred)}")

print("\n\n")
print(f"Accuracy score   {accuracy_score(y_test , pred)}")

Classification Repport
                precision    recall  f1-score   support

    Non-Spam       0.98      0.99      0.98       162
        Spam       0.93      0.87      0.90        30

    accuracy                           0.97       192
   macro avg       0.95      0.93      0.94       192
weighted avg       0.97      0.97      0.97       192




Confudion Matrix  
 [[160   2]
 [  4  26]]



Accuracy score   0.96875


In [23]:
#randomclassifier model
from sklearn.ensemble import RandomForestClassifier

clf2 = RandomForestClassifier()


In [24]:
#cross validation for random classifeir
from sklearn.model_selection import cross_val_score
valid = cross_val_score(estimator = clf2 , X =text , y=y , cv=10)
print("Cross Validation results for 10 folds")
print(valid)
print(f"Mean for cross val is {valid.mean()}")
print(f"std for the cross val is {valid.std()}")

Cross Validation results for 10 folds
[0.94791667 0.9375     0.98958333 0.96875    0.92708333 0.91666667
 0.92708333 0.95789474 0.94736842 0.93684211]
Mean for cross val is 0.945668859649123
std for the cross val is 0.020724975013174485


In [25]:
clf2.fit(x_train ,y_train)

RandomForestClassifier()

In [26]:
#checking score based on train set
from sklearn.metrics import classification_report , accuracy_score , confusion_matrix
train_pred2 = clf2.predict(x_train)

print(f"Classification Repport\n  {classification_report(y_train , train_pred2)}")

print("\n\n")
print(f"Confudion Matrix  \n {confusion_matrix(y_train ,train_pred2)}")

print("\n\n")
print(f"Accuracy score   {accuracy_score(y_train ,train_pred2)}")

Classification Repport
                precision    recall  f1-score   support

    Non-Spam       1.00      1.00      1.00       673
        Spam       1.00      1.00      1.00        92

    accuracy                           1.00       765
   macro avg       1.00      1.00      1.00       765
weighted avg       1.00      1.00      1.00       765




Confudion Matrix  
 [[673   0]
 [  0  92]]



Accuracy score   1.0


In [27]:
#checking score based on set test
pred1 = clf2.predict(x_test)

print(f"Classification Repport\n  {classification_report(y_test , pred1)}")

print("\n\n")
print(f"Confudion Matrix  \n {confusion_matrix(y_test , pred1)}")

print("\n\n")
print(f"Accuracy score   {accuracy_score(y_test , pred1)}")

Classification Repport
                precision    recall  f1-score   support

    Non-Spam       0.92      1.00      0.96       162
        Spam       1.00      0.50      0.67        30

    accuracy                           0.92       192
   macro avg       0.96      0.75      0.81       192
weighted avg       0.93      0.92      0.91       192




Confudion Matrix  
 [[162   0]
 [ 15  15]]



Accuracy score   0.921875


In [28]:
#lets check for boosting algo
from xgboost import XGBClassifier
clf3 = XGBClassifier()

In [29]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train_le = le.fit_transform(y_train)
y_test_le = le.fit_transform(y_test)

In [30]:
#cross validation
from sklearn.model_selection import cross_val_score
valid = cross_val_score(estimator = clf3 , X =text , y=y , cv=10)
print("Cross Validation results for 10 folds")
print(valid)
print(f"Mean for cross val is {valid.mean()}")
print(f"std for the cross val is {valid.std()}")

Cross Validation results for 10 folds
[nan nan nan nan nan nan nan nan nan nan]
Mean for cross val is nan
std for the cross val is nan


In [31]:
clf3.fit(x_train , y_train_le)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [32]:
#checking score based on train set
from sklearn.metrics import classification_report , accuracy_score , confusion_matrix
train_pred3 = clf3.predict(x_train)

print(f"Classification Repport\n  {classification_report(y_train_le , train_pred3)}")

print("\n\n")
print(f"Confudion Matrix  \n {confusion_matrix(y_train_le , train_pred3)}")

print("\n\n")
print(f"Accuracy score   {accuracy_score(y_train_le , train_pred3)}")

Classification Repport
                precision    recall  f1-score   support

           0       1.00      1.00      1.00       673
           1       1.00      0.97      0.98        92

    accuracy                           1.00       765
   macro avg       1.00      0.98      0.99       765
weighted avg       1.00      1.00      1.00       765




Confudion Matrix  
 [[673   0]
 [  3  89]]



Accuracy score   0.996078431372549


In [33]:
#checking score based on set test
pred3 = clf3.predict(x_test)

print(f"Classification Repport\n  {classification_report(y_test_le , pred3)}")

print("\n\n")
print(f"Confudion Matrix  \n {confusion_matrix(y_test_le , pred3)}")

print("\n\n")
print(f"Accuracy score   {accuracy_score(y_test_le , pred3)}")

Classification Repport
                precision    recall  f1-score   support

           0       0.94      0.99      0.96       162
           1       0.91      0.67      0.77        30

    accuracy                           0.94       192
   macro avg       0.93      0.83      0.87       192
weighted avg       0.94      0.94      0.93       192




Confudion Matrix  
 [[160   2]
 [ 10  20]]



Accuracy score   0.9375
